<table border="1" cellpadding="5" cellspacing="0" width="100%">
   <tr>
      <!--<td align="left" valign="top" width="120"><img src="./pictures/octopeek-logo.png" width="120" /></td>-->
      <td align="left" valign="top" width="120"><img src="https://media-exp1.licdn.com/dms/image/C510BAQE93sqc09g7qg/company-logo_200_200/0?e=2159024400&v=beta&t=Ebcbl-_mVoiGn-jo8xRX3V0iuAckEhZXnLKcmnOv2Wk" width="120" /></td>
      <td valign="top" width="100%" align="center"><h1><font color="blue">Enseignement d'Intégration - Sujet Octopeek</font></h1>
      <p align="right"><h2>Analyse de sentiments sur des Tweets</h2></p>
   </tr>
</table>

# Entrainement de modèles

Ce notebook a pour objectif de chercher un modèle et une représentation des données optimals ainsi que d'entrainer un modèle final et d'inférer la polarité des tweets de votre projet. Pour la répartition des tâches, n'hésitez pas à remplir le notebook chacun de votre coté, en partageant les fichiers de données, et rassemblez le code dans un notebook final. Vous pouvez dupliquer le notebook sur votre serveur Jupyter puis ne laisser que le final en fin de projet.

<div class="alert alert-block alert-success">
Toutes les tâches que vous avez sont parallélisables mais également peuvent se faire "partiellement" pour que le projet avance. Je donne 2 exemples :<br/>

<ul><li>Pour la partie 02-datacollect, vous pouvez créer un premier fichier pickle avec 10 000 tweets, puis vous l'agrandissez au fur et à mesure dans la semaine.</li>
<li>Pour la partie 04-models, vous pouvez faire un premier modèle qui fonctionne bien et ainsi inférer les polarités de vos tweets projet, vous pouvez ainsi avancer sur la partie 05-analysis. En parrallèle vous cherchez un meilleur modèle pour le challange en faisant d'autres combinaisons et des recherches d'hyperparamètres. Vous pouvez aussi recommencer votre pipeline de généraion de visuels dans 05-analysis avec de nouvelles prédictions plus précises.</li></ul>

L'idée est de commencer les analyses jeudi après-midi maximum. Cette gestion du projet n'est pas simple mais est un bon exercice.
</div>

## Représentation des données

La première étape est de transformer les données afin que les modèles *ML* que vous aurez choisi puissent les lire. Il s'agit souvent de transformer les documents en un vecteur représentatif de taille commune. Implémentez 2 ou 3 représentations différentes de vos données. Nous vous conseillons la représentation *TFIDF* ainsi qu'une représentation distributionnelle (Doc2Vec, moyenne des vecteur Word2Vec ou GloVe, inférence d'un modèle pré-entrainé disponible sur internet...). Vous pouvez en trouver une autre et même en choisir une a priori sous-optimale pour comparer. Ayez toujours le réflexe de chercher des bibliothèques avant de coder votre représentation *from scratch*. Si vous choisissez un modèle tel que Doc2Vec, nous vous conseillons d'effectuer un apprentissage non-supervisé avec les données que nous vous fournissons dans le fichier `tweets-for-unsupervised-training.pickle`.

Vous décrirez les différents modèles, leur avantages et inconvénients ainsi que vos choix dans le rapport et la présentation.

In [64]:
# Ouverture de vos données d'entrainement (fichier `group<X>_<dataname>_preprocessed.pickle`) :
!pip install nltk
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [2]:

import pickle
with open('group12_IMBD_preprocessed.pickle', 'rb') as f:
    l = pickle.load(f)
print(l[:100])


[{'text': "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due

In [3]:
# Entrainement non-supervisé OU "fit" d'un modèle de représentation OU chargement d'un modèle pré-entrainé :
sentiment_target_names = {1 :'positive', -1:'negative'}

def article_tokenize_simple(text):
    if type(text)!= str:
        raise Exception("The function takes a string as input data")
    else:
        tokens=text.split()  
        return tokens 
    
tokenized_collection=[]
tokenized_collection_dict={}
polarity_vector=[]
texts=[]


for i in range(1000):
    t = l[i]['text']
    
    t=article_tokenize_simple(t)
    d={}
    d['doc_id']=i
    d['tokens']= t
    d['polarity']=l[i]['polarity']
    tokenized_collection.append(d)
    tokenized_collection_dict[i]=t
    polarity_vector.append(l[i]['polarity'])
    texts.append(l[i]['text'])
    
    
sentiment_target = polarity_vector


In [4]:
voc = []
for a in tokenized_collection_dict :
    voc = voc + tokenized_collection_dict[a]
voc = set(voc)

print (len(voc))




34724


In [5]:
import numpy as np

In [6]:
from collections import OrderedDict


def build_inverted_index(collection):
    # On considère ici que la collection est pré-traitée
    inverted_index=OrderedDict()
    for document in collection:
        for term in collection[document]:
            if term in inverted_index.keys():
                if document in inverted_index[term].keys():
                    inverted_index[term][document] = inverted_index[term][document] + 1
                else:
                    inverted_index[term][document]= 1
            else:
                inverted_index[term]=OrderedDict()
                inverted_index[term][document]=1
                    
    return inverted_index

In [7]:
inverted_index=build_inverted_index(tokenized_collection_dict) 

In [8]:
#Modéle vectoriel

from math import log

def indice_tf (term, id, index) :
    tf=0
    for key, value in index[term].items() :
        if key == id :
            tf=value
            break
    return tf

def indice_idf(index,N) :
    p = {}
    for i in index :
        df = len(index[i])
        p[i]=log((N/df), 10)
    return p




def tf_idf (tf, idf) :
    return tf * idf




In [9]:
def vectorisation (index, tokenized_collection_dict) :
    d = {}
    N = len(tokenized_collection_dict)
    idf = indice_idf(index,N)
    for id in tokenized_collection_dict :
        count_vect = []
        for term in index :
            tf = indice_tf(term, id, index)
            count_vect.append(tf_idf(tf,idf[term]))
        d[id] = count_vect
    return d

In [9]:
print(type(vectorisation(inverted_index,tokenized_collection_dict)))

<class 'dict'>


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(texts)
X_train_counts.shape


(1000, 17922)

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(1000, 17922)

In [12]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


(1000, 17922)

In [13]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, sentiment_target)

In [14]:
tweets_non_labellises = ['Film is good', 'Film is really really bad']
X_new_counts = count_vect.transform(tweets_non_labellises)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

prediction = clf.predict(X_new_tfidf)

for doc, category in zip(tweets_non_labellises, prediction):
    print('%r => %s' % (doc, sentiment_target_names[category]))

print(np.mean(prediction==sentiment_target))

'Film is good' => positive
'Film is really really bad' => negative
0.0


<ipython-input-14-364cc483d139>:10: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  print(np.mean(prediction==sentiment_target))


In [15]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC())

In [16]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [17]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)



<div class="alert alert-block alert-success">
Les méthodes de traitement automatique du langage naturel dites "distributionelle" s'appuient sur l'analyse de grands corpus (ensemble de documents) dans l'objectif d'extraire de la connaissance du texte selon la distribution de chaque mots par rapport aux autres.
</div>
<div class="alert alert-block alert-success">
La machine n'a, contrairement aux humains, aucune connaissance du langage, elle n'est pas capable de trouver de proximité sémantique entre des mots comme "professeur" et "enseignant". Les lettres que composent ces mots n'apportent aucune information sans connaissance du monde. Une première méthode pour créer des liens entre notions est de créer manuellement un dictionnaire de synonymes, d'ontologies ou autres. Mais la tâche est complexe car le langage est subtil et évolue dans le temps, il est donc plus facile d'inférer des règles ou des relations automatiquement que d'essayer manuellemnt de décrire le langage de facon exhaustive. Les méthodes distributionelles permettent d'exploiter de grands corpus pour automatiquement inférer des similarités entre mots.
</div>
<div class="alert alert-block alert-success">
La sémantique distributionelle se fonde sur l'hypothèse de Harris énnoncé en 1954 : les mots ayant statistiquemet le même contexte seront plus probablement similaire sémantiquement. Concretement, cela signifie que les mots ayant les mêmes voisins dans les phrases sont souvent sémantiquement proches. Par exemple les mots "voiture" et "moto" sont souvent voisins du verbe "rouler", on estimera alors que "moto" et "voiture" sont sémantiquement proches.
</div>
<div class="alert alert-block alert-success">
Les modèles tel que Word2Vec font en sorte que les vecteurs attribués à "moto" et "voiture" soient proches d'un point de vue similarité (cosinus...). Pouvoir calculer des simlarités entre mots est important dans la compréhension du langage puisque permet de situer chaque notion par rapport aux autres et ainsi décrire une notion par ses notions proches. Avec les méthodes distributionnelles, les vecteurs de mots sont placés dans un espace vectoriel qui a du sens. Plus recemment, des outils tel que Doc2Vec ou le Deep Learning permettent de représenter des phrases et des documents ayant plus de sens que la simple moyenne des mots.
</div>

# Analyse qualitative de tweets similaires et dissimilaires

Pour chaque modèle de représentation, prennez 10 tweets et affichez le tweets le plus similaire et celui le plus dissimilaire. Vous sauterez une ligne entre chaque triplet. Utilisez par exemple la similarité cosinus. Si le temps de calcul est long, ne comparez un exemple qu'à seulement 1000 autres. Faites une analyse qualitative, commentez.

In [18]:
#Analyse qualitative
d=vectorisation(inverted_index,tokenized_collection_dict)
echantillons_tweets = texts[:10]
tweets_de_comparaison = texts[10:1010]

def similarite_cosinus(tweet, liste):
    maxi, mini = -1, 1
    vect_maxi = 'Pas de tweet similaire'
    vect_mini = 'Pas de tweet dissimilaire'
    vect_tweet = d[tweet]
    for comparatif in list:
        vect_compa = d[comparatif]
        prod_scal, norm_carre_tw, norm_carre_comp = 0,0,0
        for i in range(len(l)):
            prod_scal+=vect_tweet[i]*vect_compa[i]
            norm_carre_tw, norm_carre_comp = norm_carre_tw + vect_tweet[i]^2, norm_carre_comp + vect_compa[i]^2
        cosin = prod_scal/sqrt(norm_carre_tw*norm_carre_comp)
        if cosin>maxi:
            maxi=cosin
            vect_maxi = comparatif
        if cosin<mini:
            mini=cosin
            vect_mini = comparatif
    return ('Tweet le plus dissimilaire : ', vect_mini, '\n', 'Tweet le plus similaire : ', vect_maxi)

for j in range(len(echantillons_tweets)):
    (a,b)=similarite_cosinus(echantillons_tweets[j],tweets_de_comparaison)
    print(echantillons_tweets[j][:200], '\n Tweet le plus dissimilaire : ', a[:200], '\n', 'Tweet le plus similaire : ', b[:200])
    


KeyError: "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due

<div class="alert alert-block alert-success">
Une analyse qualitative consiste en l'affichage de quelques exemples afin d'observer que ce que vous calculez correspond à ce que vous attendez. Si vous observez des résultats inattendus, alors votre code est peut-être à revoir.
</div>

## Implémentation des modèles

Choisissez 2 modèles minimum dans Scikit-learn. Vous pouvez vous aider de [ce graphe](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) pour choisir. Vous pouvez aussi implementer des modèles avec d'autres librairies si vous pensez avoir le temps.

Faites une 10-fold cross-validation de toutes les combinaisons représentation des données / modèles. Pour chaque combinaison vous devez choisir quelques hyperparamètres et effectuer un grid-search pour trouver les meilleurs hyperparamètres. Falcutativement vous pouvez aussi inclure des hyperparamètres qui permettent la génération des représentations (par exemple `min_df` pour *TFIDF*). Vous devez donc utiliser un outil de Scikit-learn qui gère à la fois le grid-search et la cross-validation. Utilisez la métrique *accuracy*. Pour chaque combinaison, affichez le score cv moyen ainsi que les meilleurs hyperparamètres.

Vous décrirez les différents modèles, leur avantages et inconvénients ainsi que vos choix dans le rapport et la présentation.

In [10]:
# Initialisation des variable `X` et `y` :
x1 = vectorisation(inverted_index, tokenized_collection_dict)
X,y = [],[]
y = []
for id in x1:
    X.append(x1[id])
    y.append(tokenized_collection[id]['polarity'])

print(X[:6])
print(y[0:6])



[[1.1487416512809248, 0.1463496955317154, 0.07955986262508613, 1.2006525570379236, 2.0969100130080562, 0.4089353929735008, 1.6020599913279623, 0.45403709931007247, 0.7166987712964503, 0.8068754016455384, 0.7210270214628279, 1.2518119729937995, 5.999999999999999, 2.8178707859470014, 1.3565473235138124, 0.5208553110998161, 1.9586073148417746, 1.1611509092627446, 0.5432924359575431, 2.221848749616356, 0.8873949984654255, 0.16079795243177805, 0.437962330836741, 1.4814860601221125, 0.973564799864122, 1.6197887582883939, 0.7745097999287158, 1.6173064717303562, 0.3298877965597676, 1.2879482856137543, 0.8860566476931632, 4.0, 0.3798639450262425, 0.5794259114435477, 0.6946486305533761, 2.6989700043360183, 0.10365967398872758, 2.6989700043360183, 0.8356471442155629, 3.772113295386326, 0.4749551929631547, 1.1366771398795439, 0.17397584093306018, 1.0506099933550872, 0.3381873144627387, 2.655804284128565, 2.397940008672037, 2.5228787452803374, 1.4436974992327125, 0.5129804705431404, 0.0518298369943

In [ ]:
# Grid-search :


In [ ]:
# Affichage des meilleurs paramètres et score cv moyen de chaque combinaison :


<div class="alert alert-block alert-success">
Il est toujours conseillé d'implementer tout d'abord avec un petit échantillon de vos données afin que les évaluations soient rapides. Vous utiliserez ensuite l'ensemble des données quand tout fonctionnera pour avoir les vrais scores.
</div>

Faites une analyse qualitative des prédictions de 30 exemples avec le meilleur modèle accessible avec l'attribut `best_estimator_`

In [ ]:
# Analyse qualitative :


## Points bonus facultatifs - Librairie *TextBlob*

Cette étape peut facilement être faite en parallèle dans un autre notebook. Vous remettrez votre code fonctionnel dans ce notebook. Utilisez la librairie *TextBlob* pour la prédiction de polarité des tweets. *TextBlob* est en quelques sorte un modèle pré-entrainé qui renvoie une polarité à partir d'un texte brut. Vous devez effectuer la même cross-validation (mêmes paramètres, notamment `cv`) en utilisant [cross_validate](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) et un [estimator custom](https://stackoverflow.com/questions/20330445/how-to-write-a-custom-estimator-in-sklearn-and-use-cross-validation-on-it)) et afficher le score cv moyen. Bien entendu les données ne seront pas les textes encodés mais les textes bruts.

In [ ]:
# Installatin de la librairie :
!pip install textblob

In [ ]:
# Votre code :


## Tableau récapitulatif

Créez un tableau récapitulatif en html ou latex (via markdown) rassemblant toutes les combinaisons avec les colonnes :

 * **Data-repr** (representation des données)
 * **Model**
 * **Hyperparameters** (meilleurs hypeparamètres en format court)
 * **Accuracy** (moyenne des scores cv pour votre métrique accuracy ou autre)

Ajoutez *TextBlob* si vous l'avez implementé.

In [ ]:
### Tableau (changez le type de cellule) :


## Entrainement du meilleur modèle sur toutes les données

Une fois que vous avez trouvé le meilleur modèle (avec les meilleurs hyperparamètres) et la meilleur représentation, il est important de disposer d'un modèle prédictif ayant toute les chances de généraliser et d'être précis. Pour cela la dernière étape d'un projet ML est d'entrainer le modèle sur toutes les données que nous disposons. Cette étape est souvent oubliée dans la recherche qui se concentre sur le scoring et la comparaison des modèles. Entrainez un modèle sur toutes les données et sauvegardez le. Vous le nommerez `group<X>_<datarepr>_<model>_finalmodel.pickle`.

In [ ]:
# Entrainement :



In [ ]:
# Sauvegarde :


## Inférence de la polarité sur vos données

Reprennez votre fichier `group<X>_<dataname>_projectdata.pickle` de données contenant tous vos tweets. Ce fichier doit normalement contenir une liste de dictionnaires avec les champs `id` et `text` pour chaque élément. Le champs `text` doit correspondre au texte brut non transformé. Vous aurez bien entendu ajouté d'autres champs qui vous sont utiles comme une catégorie ou les mots clefs de requête par exemple.


Pour cette étape vous devez créer une fonction `preprocess(text)` qui reprend tout le pipeline de prétraitement du texte brut. Pour chaque tweet, prétraitez son texte brut, puis inférez la polarité avec votre modèle pré-entrainé. Ajouter le champs `predicted_polarity` et re-sérialisez les données sous le même nom de fichier.

In [ ]:
# Désérialisation de votre modèle final :

In [ ]:
# Fonction `preprocess` :


<div class="alert alert-block alert-success">
Certaines étapes de prétraitement sont considérés comme des modèles dans Scikit-learn, vous pouvez alors réutiliser ces "modèles" et leur méthode transform afin d'effectuer une transformation de nouveaux exemples. C'est par exemple le cas de TfidfVectorizer.
</div>

In [58]:
# Lecture des données :
# Ouverture du fichier pickle :
import pickle
with open('group12_electriccars.projectdata.pickle', 'rb') as f:
    test_tweets = pickle.load(f)
print(type(test_tweets))

<class 'list'>


In [62]:
for i in range(len(test_tweets)) :
    test_tweets[i] = retirer_accents(test_tweets[i])
    test_tweets[i] = retirer_liens(test_tweets[i])
    test_tweets[i] = retire_break(test_tweets[i])  
    test_tweets[i] = retirer_caracteres_speciaux(test_tweets[i])

In [63]:
# Ajout du champs `predicted_polarity` :

docs_new = test_tweets
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, sentiment_target_names[category]))




'Total global sales of new electric car will reach 34  by 2030 and 68  by 2040 ' => negative
'@BrightonHoveCC @BHCC Transport @EBChargingLtd @electricbton There is a charging point in Holland Road #hove it is hardly ever available to electric car drivers as it is made available to non electric cars who park in it, often for complete days of weekends  Why charging points not exclusive to electric vehicles I\n@BHGreenCllrs @CllrEbel' => negative
'We’re inviting Expressions of Interest from Fingal based businesses who would like to trial delivery using eCargo bikes  Get access to electric or eCargo bikes  amp  replace your delivery or service trips normally taken by car van  \n' => negative
"🚗 At EVEE, trust and security matters  We're committed to building a community of car owners and EV renters \n\nWe now offer comprehensive insurance solutions and options for Tesla and other electric car owners and renters  👀\n\nCome see for yourself  👇\n " => negative
"Tech Update  Apple hires former

In [ ]:
# Re-sérialization du fichier :


Faites une analyse qualitative de 30 tweets afin de voir si votre modèle prédit des labels qui vous semblent corrects.

In [ ]:
# Analyse qualitative :


## Partie challenge

Pour la partie challenge, nous vous demandons de prendre un fichier `pickle` contenant une liste de tweets et de nous envoyer la polarité de chacun des tweets dans le même ordre. Implémentez ces différentes étapes pour le fichier `trialset.pickle` (jeu de test trial) que vous trouverez sur le repo github. Attention vous devez respecter l'ordre des tweets pour que vos prédictions correspondent à nos labels.

Il vous sera demandé, dans un temps très court, d'effectuer les mêmes étapes pour notre vrai fichier d'évaluation (jeu de test) en fin de semaine contenant 9000 exemples. Vous devez donc vous assurer que tout fonctionne sur cette partie du notebook. Après la création de votre liste de polarité pickle sur notre jeu de test, vous l'enverrez au responsable de formation. Vous nommerez le fichier `group<X>_testset_predictions.pickle`.

In [59]:
# Ouverture du fichier pickle :
import pickle
with open('testset.pickle', 'rb') as f:
    test_texts = pickle.load(f)
    


<class 'list'>


In [33]:
# Prétraitement de tous les tweets :


import unicodedata, re


def retirer_liens(s) :
  return re.sub(r"http\S+", "", s)

def retirer_caractere_speciaux(s) :
  caracteres_speciaux ='~!$%^&*()_-+={}\/:;<>?.'
  for i in caracteres_speciaux :
     s = s.replace(i, ' ')
  return s



def retirer_break(s):
    
    return s.replace("<br /><br />", " ")


def retirer_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')
    return s

for i in range(len(test_texts)-1) :
    test_texts[i] = retirer_accents(test_texts[i])
    test_texts[i] = retirer_liens(test_texts[i])
    test_texts[i] = retirer_break(test_texts[i])  
    test_texts[i] = retirer_caracteres_speciaux(test_texts[i])
    
print(test_texts[0:3])

hola   que tal


In [41]:
# Inférence des polarités :
docs_new = test_texts
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, sentiment_target_names[category]))


" Price  I'd take Vitali fight  David Price says he would fight Vitali K     CLICK 1st LINK  2WIN" => negative
'Kim Hyung Jun   Football Team the 2nd A Match at YeongDeungPo gu DaeRimDong [12 10 27]\nCredit   tlsxhah\n#8 ' => positive
"Queens of the Stone Age's Regular John from Glastonbury during the Song For The Deaf tour  amp  Dave Grohl sat on the stool " => positive
'All my thoughts and prayers go out to Sergei Bubka Jr  who fell from the 3rd floor apartment in Paris ' => positive
'Crimes against women  Khaps, women panel meeting on Oct 28  It would be for the first time that Haryana khap panchayats and National    ' => negative
"the only thing i may truly miss about lexington is shoto's    " => positive
"@waffl3lif3 @keyvo33 ya for sure it's on 28th and Ost starts at 1043 but really not till 11 lol" => negative
"Wayne is on his 4th single w  Carter IV already   still no album drop   Sounds to me like he's searching for hits but hasn't found one yet" => positive
'Kevin Rudd talks 

In [44]:
# Sérialisation des polarités (liste d'entier -1, 0, 1 de même taille que le jeu de test) :
print(type(predicted))
print(predicted)

<class 'numpy.ndarray'>
[-1  1  1  1 -1  1 -1  1  1  1  1 -1  1 -1  1  1  1 -1 -1  1 -1 -1 -1 -1
 -1  1  1 -1  1 -1 -1  1 -1  1  1 -1  1 -1  1 -1  1 -1  1  1  1  1 -1 -1
 -1 -1  1  1  1  1  1 -1  1 -1  1 -1 -1 -1  1  1  1 -1 -1 -1  1 -1  1 -1
 -1  1 -1  1 -1  1  1  1 -1  1  1  1 -1  1 -1  1  1 -1  1 -1 -1 -1 -1 -1
 -1  1  1 -1  1 -1  1 -1  1  1  1 -1 -1 -1  1  1  1 -1  1 -1  1 -1 -1  1
 -1  1 -1 -1 -1 -1 -1 -1  1  1 -1  1 -1  1  1 -1  1 -1 -1 -1 -1 -1 -1  1
  1 -1 -1 -1 -1  1 -1 -1  1 -1  1 -1  1  1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1
 -1 -1 -1  1  1 -1  1 -1 -1  1 -1 -1 -1  1 -1  1  1 -1 -1  1 -1 -1  1 -1
  1 -1  1  1 -1  1  1 -1 -1  1 -1 -1  1 -1 -1 -1 -1 -1  1  1 -1  1 -1  1
 -1 -1 -1 -1 -1  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1  1
  1 -1 -1 -1 -1  1  1  1  1  1 -1  1  1 -1 -1 -1 -1  1 -1  1 -1 -1 -1 -1
 -1  1 -1 -1 -1  1 -1 -1  1  1  1  1  1 -1 -1 -1 -1  1  1 -1  1  1  1  1
 -1 -1 -1 -1  1  1 -1  1  1 -1 -1  1 -1 -1  1  1  1 -1 -1  1 -1  1 -1 -1
  1  1  1 -1 -1 -1  1  1  1

In [46]:

import pickle
file_Name = "g12_testset_predictions.pickle"
# open the file for writing
fileObject = open(file_Name,'wb')

pickle.dump(predicted,fileObject) 

fileObject.close()

## Suite du projet

Vous devez maintenant analyser vos données de projet dans un nouveau notebook qui reprendra votre fichier de données avec les prédictions de polarité. Vous le nommerez `05-analysis.ipynb`.

 1. Utilisez <a href="https://docs.bokeh.org/en/latest/index.html">Bokeh</a> ou <a href="https://matplotlib.org/">Matplotlib</a> pour générer des graphes, courbes. Utilisez d'autres librairies pour produire d'autres visuels tels que des nuages de mots.
 2. Vous pouvez aussi générer quelques statistiques.
 3. Tirer des conclusions à valeur ajoutée, répondez à des interogations comme si vous adressiez à un client, soyez original.
 4. Présentez d'autres analyses qui aurait pu être implémentées avec plus de temps.